In [47]:
from robot_configs.test import grid_to_rewards,coordinate_finder,epsilon,generate_episode,get_move_from_policy
import pickle
import os
import numpy as np
from environment import Robot

In [48]:
with open(os.path.join('grid_configs',"house.grid"), 'rb') as f:
            grid = pickle.load(f)
robot = Robot(grid, (1, 1), orientation='n', battery_drain_p=0.5, battery_drain_lam=2, p_move=0.0)
move_pairs = list(robot.dirs.items())

In [49]:
def behavior_policy(robot):
    iter=0
    # (move_orientation, (move_increment)) pairs
    move_pairs = list(robot.dirs.items())
    # no of rows
    n_rows = robot.grid.n_rows
    # no of cols
    n_cols = robot.grid.n_cols
    # Initialization of e-soft policy, Q and returns
    Q = np.zeros((n_cols, n_rows, 4)).tolist()
    rewards = grid_to_rewards(robot)
    policy = ((np.ones((n_cols, n_rows, 4)) * epsilon) / 4).tolist()
    for x in range(1, n_cols - 1):
        for y in range(1, n_rows - 1):
            # Action that maximizes Q(s,a)
            max_action_idx = Q[x][y].index(max(Q[x][y]))
            for action_id in range(4):
                move_orientation, _ = move_pairs[action_id]
                (next_x, next_y) = coordinate_finder((x, y), move_orientation)
                # Make sure to never explore moves into walls/obstacles
                if rewards[next_x][next_y] == -1:
                    #pi(a|s)
                    policy[x][y][action_id] = 0
                elif max_action_idx == action_id:
                    policy[x][y][max_action_idx] = 1 - epsilon + epsilon / 4
                else:
                    policy[x][y][action_id] = epsilon / 4
        iter += 1
    return policy


In [50]:
behavior_policy = behavior_policy(robot)
#1.
ep_history, ep_return = generate_episode(behavior_policy, robot)

Simulated moves:  400


In [52]:
print(ep_history,ep_return)

deque([(1, 1, 'e'), (2, 1, 'e'), (3, 1, 'e'), (4, 1, 'w'), (3, 1, 'w'), (2, 1, 'e'), (3, 1, 's'), (3, 2, 'n'), (3, 1, 's'), (3, 2, 'n'), (3, 1, 'w'), (2, 1, 's'), (2, 2, 'n'), (2, 1, 'e'), (3, 1, 's'), (3, 2, 'n'), (3, 1, 'w'), (2, 1, 'w'), (1, 1, 's'), (1, 2, 'n'), (1, 1, 's'), (1, 2, 'n'), (1, 1, 's'), (1, 2, 'n'), (1, 1, 'e'), (2, 1, 'e'), (3, 1, 's'), (3, 2, 'e'), (4, 2, 'n'), (4, 1, 'e'), (5, 1, 'e'), (6, 1, 'w'), (5, 1, 'e'), (6, 1, 's'), (6, 2, 'n'), (6, 1, 'w'), (5, 1, 'w'), (4, 1, 'w'), (3, 1, 'w'), (2, 1, 's'), (2, 2, 'n'), (2, 1, 's'), (2, 2, 'n'), (2, 1, 's'), (2, 2, 'n'), (2, 1, 'e'), (3, 1, 's'), (3, 2, 'n'), (3, 1, 'w'), (2, 1, 'w'), (1, 1, 'e'), (2, 1, 's'), (2, 2, 'n'), (2, 1, 'e'), (3, 1, 'w'), (2, 1, 'e'), (3, 1, 's'), (3, 2, 'n'), (3, 1, 's'), (3, 2, 'n'), (3, 1, 'w'), (2, 1, 'e'), (3, 1, 'w'), (2, 1, 's'), (2, 2, 'w'), (1, 2, 'n'), (1, 1, 'e'), (2, 1, 's'), (2, 2, 'n'), (2, 1, 's'), (2, 2, 'n'), (2, 1, 'e'), (3, 1, 'e'), (4, 1, 'w'), (3, 1, 'e'), (4, 1, 's'), (4, 2

In [12]:
#2.
def backwards(ep_history,target_policy):
    # state_action_episode_end = ep_history[-1]
    # state_episode_end = (state_action_episode_end[0],state_action_episode_end[1])
    #compare from a_T or a_T-1 (currently from T-1)
    for i in range(1,len(ep_history)+1):
        state = (ep_history[-1-i][0],ep_history[-1-i][1])
        action_behavior = ep_history[-1-i][2]
        #action_target = get_move_from_policy(policy,state)
        #print(action_behavior)
        action_target, move_increment = move_pairs[get_move_from_policy(target_policy,state)]
        #print(action_target)
        if(action_behavior!=action_target):
            break
    return i


In [13]:
t = len(ep_history)-backwards(ep_history,target_policy)
t

395

In [10]:
def target_policy(robot):
    iter=0
    # (move_orientation, (move_increment)) pairs
    move_pairs = list(robot.dirs.items())
    # no of rows
    n_rows = robot.grid.n_rows
    # no of cols
    n_cols = robot.grid.n_cols
    # Initialization of e-soft policy, Q and returns
    Q = np.zeros((n_cols, n_rows, 4)).tolist()
    rewards = grid_to_rewards(robot)
    policy = ((np.ones((n_cols, n_rows, 4)) * epsilon) / 4).tolist()
    for x in range(1, n_cols - 1):
        for y in range(1, n_rows - 1):
            # Action that maximizes Q(s,a)
            max_action_idx = Q[x][y].index(max(Q[x][y]))
            for action_id in range(4):
                move_orientation, _ = move_pairs[action_id]
                (next_x, next_y) = coordinate_finder((x, y), move_orientation)
                # Make sure to never explore moves into walls/obstacles
                if rewards[next_x][next_y] == -1:
                    #pi(a|s)
                    policy[x][y][action_id] = 0
                elif max_action_idx == action_id:
                    policy[x][y][max_action_idx] = 1 - epsilon + epsilon / 3
                else:
                    policy[x][y][action_id] = epsilon / 3
        iter += 1
    return policy

In [11]:
target_policy = target_policy(robot)

In [39]:
#3.
def pi_target_behavior(t):
    multi = 1
    for i in range(t-1,len(ep_history)-1):
        state = (ep_history[i][0],ep_history[i][1])
        action = ep_history[i][2]
        if action == 'n': action= 0
        elif action == 'e': action = 1
        elif action == 's': action= 2
        else: action = 3
        pi_bahavior = behavior_policy[state[0]][state[1]][action]
        multi = multi*(1/pi_bahavior)
    return multi

In [24]:
move_pairs = list(robot.dirs.items())
print(move_pairs)

[('n', (0, -1)), ('e', (1, 0)), ('s', (0, 1)), ('w', (-1, 0))]


In [ ]:
def calc_action_val(ep_historys):
    """
    Calculate action value based on random move probability
    Primary move probability = 1-p_move + p_move/4
    Secondary move probability = p_move/4
    """
    (x, y) = pos

    if action_orientation == 'n':
        r_move_n = ((1-random_move_p)+(random_move_p/4)) * \
            (grid_cells[x, y - 1] + gamma * vals[x, y - 1])
        r_move_s = (random_move_p/4)*(grid_cells[x, y+1]+gamma*vals[x, y+1])
        r_move_e = (random_move_p/4)*(grid_cells[x+1, y]+gamma*vals[x+1, y])
        r_move_w = (random_move_p/4)*(grid_cells[x-1, y]+gamma*vals[x-1, y])
    elif action_orientation == 's':
        r_move_n = (random_move_p / 4) * \
            (grid_cells[x, y - 1] + gamma * vals[x, y - 1])
        r_move_s = ((1 - random_move_p) + (random_move_p / 4)) * \
            (grid_cells[x, y + 1] + gamma * vals[x, y + 1])
        r_move_e = (random_move_p / 4) * \
            (grid_cells[x + 1, y] + gamma * vals[x + 1, y])
        r_move_w = (random_move_p / 4) * \
            (grid_cells[x - 1, y] + gamma * vals[x - 1, y])
    elif action_orientation == 'e':
        r_move_n = (random_move_p / 4) * \
            (grid_cells[x, y - 1] + gamma * vals[x, y - 1])
        r_move_s = (random_move_p / 4) * \
            (grid_cells[x, y + 1] + gamma * vals[x, y + 1])
        r_move_e = ((1 - random_move_p) + (random_move_p / 4)) * \
            (grid_cells[x + 1, y] + gamma * vals[x + 1, y])
        r_move_w = (random_move_p / 4) * \
            (grid_cells[x - 1, y] + gamma * vals[x - 1, y])
    elif action_orientation == 'w':
        r_move_n = (random_move_p / 4) * \
            (grid_cells[x, y - 1] + gamma * vals[x, y - 1])
        r_move_s = (random_move_p / 4) * \
            (grid_cells[x, y + 1] + gamma * vals[x, y + 1])
        r_move_e = (random_move_p / 4) * \
            (grid_cells[x + 1, y] + gamma * vals[x + 1, y])
        r_move_w = ((1 - random_move_p) + (random_move_p / 4)) * \
            (grid_cells[x - 1, y] + gamma * vals[x - 1, y])

    return r_move_n + r_move_s + r_move_e + r_move_w

In [53]:
ep_historys = [ep_history,ep_history]
ep_returns = [ep_return, ep_return]

In [54]:
#4.
def calc_action_val(ep_historys,ep_returns):
    sum_up = 0
    sum_down = 0
    for i in range(len(ep_historys)):
        ep_history = ep_historys[i]
        ep_return = ep_returns[i]
        t = len(ep_history)-backwards(ep_history,target_policy)
        sum_up = sum_up+pi_target_behavior(t)*ep_return
        sum_down = sum_down+pi_target_behavior(t)
    return sum_up/sum_down

In [68]:
N=0
D=0
n_rows = robot.grid.n_rows
n_cols = robot.grid.n_cols
Q = (np.zeros((n_rows, n_cols, 4))).tolist()
tao = len(ep_history)-backwards(ep_history,target_policy)
for t in range(tao-1,len(ep_history)-1):
    state = [ep_history[t][0], ep_history[t][1]]
    action = ep_history[t][2]
    if action == 'n': action= 0
    elif action == 'e': action = 1
    elif action == 's': action= 2
    else: action = 3
    W=pi_target_behavior(t)
    N = N+W*ep_return[t]
    D = D+W
    Q[state[0]][state[1]][action] = N/D
    print(W,N,D)


1600.0 0.0 1600.0


In [66]:
Q

[[[0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0]],
 [[0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0]],
 [[0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0